In [1]:
import pymongo
from pymongo import MongoClient
import time
from decimal import Decimal
import os
from ftplib import FTP
import pprint
import requests

In [2]:
client = MongoClient()
client = MongoClient('localhost', 27017)

In [3]:
db_aemet = client.AEMET
db_croja = client.CRUZROJA
col_playas = db_croja['playas']
col_localidades = db_aemet['localidades']
col_costas = db_aemet['costas']
today = time.strftime("%Y-%m-%d")

In [5]:
class FtpPublish(object):
    '''
    Publish sitemap on FTP
    
    Settings in config file:
    
    publish_method = ftp
    ftp_host = example.com
    ftp_user = user
    ftp_pass = pass
    '''

    def __init__(self, ftp_host, ftp_user, ftp_pass, ftp_dir):

        # Get Settings
        self.ftp_host = ftp_host
        self.ftp_user = ftp_user
        self.ftp_pass = ftp_pass
        self.ftp_dir = ftp_dir

        self.ftp = None

    def _connect(self):
        '''
        Connect to FTP
        '''

        self.ftp = FTP(self.ftp_host,self.ftp_user, self.ftp_pass)
        self.ftp.cwd(self.ftp_dir)

    def store(self,**kwargs):
        # Open connection
        self._connect()

        # Upload and play    

        # Upload files as .tmp
        for filename in kwargs['files']:
            f = open(filename,'rb')
            self.ftp.storbinary('STOR %s.tmp' % os.path.basename(filename), f)
            f.close()

        # Rename the .tmp files
        for filename in kwargs['files']:
            self.ftp.rename('%s.tmp' % os.path.basename(filename), '%s' % os.path.basename(filename))

        # Close connection
        self.ftp.quit()

In [7]:
#Hago una query de comprobación (saco unicamente un registro):
playas  = col_playas.find({"id_costa": { "$exists" : True }},
                             {"nombre_playa":1,
                              "id_costa":1,
                              "municipio":1,
                              "provincia":1,
                               today+".medusas": 1})

mapa = []

paths = "/path/to/xml"
docs_buscador = []

for playa in playas:
    doc = {}
    if today in playa:
        
        id_costa = str(playa["id_costa"]).zfill(7)
        
        #print "El documento de playas es (cruz roja)"
        #print playa

        #print "El documento de costas para esa playa (aemet):"
        costas = col_costas.find({"_id": unicode(id_costa)}, 
                                     {"nombre_playa":1, 
                                      "nombre_localidad":1, 
                                      "nombre_provincia":1 , 
                                      "longitud":1,
                                      "latitud":1})
        
        doc['idplaya'] = playa['_id']
        #print "############################"
        #pprint.pprint(playa)
        for costa in costas:
            doc['latitud'] = costa['latitud']
            doc['longitud'] =  costa['longitud']
            #pprint.pprint(costa)
            doc['pois'] = "%s,%s" % (costa['latitud'], costa['longitud'])

        doc['path'] = paths
        
        docs_buscador.append(doc)
        
        enlace = '<a href="#">%s</a>' % str(playa['nombre_playa'].encode('latin1'))
        mapa.append([enlace, float(Decimal(costa['latitud'])), float(Decimal(costa['longitud']))])

if len(docs_buscador) > 0:
    pprint.pprint(docs_buscador[0])
        
#len(docs_buscador)
#pprint.pprint(docs_buscador)

{'idplaya': 280,
 'latitud': u'-3.78333333333',
 'longitud': u'43.4766666667',
 'path': '/path/to/xml',
 'pois': u'-3.78333333333,43.4766666667'}


In [8]:
url_publicacion = "http://des.buscador.srv.vocento.in/krammer/playas/publish"

In [9]:
xml = '''<?xml version="1.0" encoding="utf-8"?>
<playa>
    <idplaya>%s</idplaya>
    <longitud>%s</longitud>
    <latitud>%s</latitud>
    <pois>%s</pois>
    <path>%s</path>
</playa>
'''
for doc in docs_buscador:
    xmlpublish = xml %(doc['idplaya'], doc['longitud'], doc['latitud'], doc['pois'], doc['path'])
    #print "###########################"
    #print xmlpublish
    #print "###########################"
    r = requests.post(url_publicacion, data=xmlpublish)
    print r.status_code

<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>
<type 'int'>